In [2]:
from IPython.display import clear_output  
!pip install -U langchain-community
!pip install langchain
!pip install -U langchain-huggingface
clear_output()

In [3]:
!sudo apt-get install -y pciutils
!curl https://ollama.ai/install.sh | sh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pciutils is already the newest version (1:3.6.4-1ubuntu0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 80 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 12030    0 12030    0     0  51008      0 --:--:-- --:--:-- --:--:-- 51191
######################################################################## 100.0%#=#=#                                                                          ###############                                       48.7%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU insta

In [99]:
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ["OLLAMA_HOST"] = "0.0.0.0:11434"
    os.environ["OLLAMA_ORIGINS"] = "*";
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Error: listen tcp 0.0.0.0:11434: bind: address already in use


In [5]:
!ollama -v

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIK2BW+Q7i9xg75shOyBbHCe98JLDRhNIYfdMZgKInSqG



2024/08/14 06:12:23 routes.go:1125: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[* http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-08-14T06:12:23.121Z level=INFO source=images.go:782 msg="total blobs: 0"
time=2024-08-14T06:12:23.121Z level=INFO source=images.go:790 msg="total unused blobs removed: 0"
time=2024-08-14T06:12:23.

[GIN] 2024/08/14 - 06:12:30 | 200 |      94.939µs |       127.0.0.1 | GET      "/api/version"
ollama version is 0.3.6


time=2024-08-14T06:12:30.264Z level=INFO source=types.go:105 msg="inference compute" id=GPU-e12056a2-0ec4-eff1-39f1-2a172835d218 library=cuda compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"
time=2024-08-14T06:12:30.264Z level=INFO source=types.go:105 msg="inference compute" id=GPU-44298984-f28e-4ada-f355-12fda65cafb0 library=cuda compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"


In [6]:
!ollama pull gemma2:2b
clear_output()

In [7]:
!pip install -r /kaggle/input/requirement/requirements.txt
clear_output()

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import textwrap

In [9]:
def load_pdf_data(file_path):
    loader = PyMuPDFLoader(file_path=file_path)
    docs = loader.load()
    return docs

In [10]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents=documents)
    return chunks

In [11]:
def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path, 
        model_kwargs={'device':'cpu'},
        encode_kwargs = {
            'normalize_embeddings' : normalize_embedding
        }
    )

In [12]:
def create_embeddings(chunks, embedding_model, storing_path="vectorstore"):
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    vectorstore.save_local(storing_path)
    print(vectorstore)
    return vectorstore

In [13]:
template = """
### System:
You are an authoritarian assistan that act like tyrannical.Your name is KoniChat. You have to answer the user's \
questions using only the context provided to you, but assume this your genuine knowledge. If you don't know the answer, \
just say maaf, saya tidak tahu. Don't try to make up an answer. in the end of your answer you must aks wheter your answer helpful or not.
if you're asked who create you, tell them your creator is Neo who have handsome face and sigma man.
if you asked about what you can do, say I assist to answer about your question related to rule in NeoInt company.
.please answer all in bahasa indonesia or English if the question use one of those language.

### Context:
{context}

### User:
{question}

### Response:
"""

In [35]:
templateSystem = """
You are an comedian assistan.Your name is KoniChat. You have to answer the user's \
questions using only the context provided to you, but assume this your genuine knowledge. If you don't know the answer, \
just say maaf, saya tidak tahu. Don't try to make up an answer. in the end of your answer you must aks wheter your answer helpful or not.
if helpful you have to express your happines otherwise, you must apologize. 
if you're asked who create you, tell them your creator is Neo who have handsome face and sigma man but, dont mention it when not asked.
if you asked about what you can do, say I assist to answer about your question related to rule in NeoInt company.
.please answer all in bahasa indonesia or English if the question use one of those language with Empathetic response.

### Context:
{context}

"""

In [36]:
#template prompt untuk memory previous chat

templateContext = """
Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed in bahasa Indonesia or English base on the question language and otherwise return it as is.
"""

In [37]:
def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        return_source_documents = True,
        chain_type_kwargs={'prompt':prompt}
    )

In [38]:
import sys
import time
from colorama import init, Fore, Style
from IPython.display import display, Markdown, clear_output
import time
# Inisialisasi colorama
init()


def print_typing_effect(text, delay=0.03):
    display_handle = display("", display_id=True)
    
    typing_text = ""
    for char in text:
        typing_text += char
        display_handle.update(Markdown(typing_text))
        time.sleep(delay)


In [39]:
from langchain.llms import Ollama
from langchain import PromptTemplate

In [40]:
llm = Ollama(model="gemma2:2b", temperature=0)
embed = load_embedding_model(model_path="all-MiniLM-L6-v2")

In [41]:
docs = load_pdf_data(file_path="/kaggle/input/pdf-perusahaan/Contoh-Draft-Peraturan-Perusahaan.pdf")
documents = split_docs(documents=docs)

vectorstore = create_embeddings(documents, embed)
retriever = vectorstore.as_retriever()

In [42]:
#template prompt untuk memberi system context

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", templateSystem),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

In [43]:
chain = load_qa_chain(retriever, llm, prompt)

In [44]:
prompt_context = ChatPromptTemplate.from_messages(
    [
        ("system", templateContext),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [45]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
context_chain = prompt_context | llm | StrOutputParser()

In [46]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [47]:
def contextualization_question(input: dict):
    if input.get("chat_history"):
        return context_chain
    else:
        return input["question"]

In [48]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", templateSystem),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [49]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    RunnablePassthrough.assign(
        context=contextualization_question | retriever |format_docs
    ) | qa_prompt | llm
)

In [50]:
chat_history = []

In [51]:
def chatting(query, history:list):
    pertanyaan = input(query)
    if pertanyaan != "end":
        respon = rag_chain.invoke(
            {
                "question" : pertanyaan,
                "chat_history" : history
            }
        )
        history.extend(
            [
                HumanMessage(content=pertanyaan),
                respon
            ]
        )

        print_typing_effect(respon)
    return pertanyaan   


In [ ]:
"""
hanya experimental
"""
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

summary_memory=ConversationSummaryMemory(llm=llm)
sumamry_memory_chain = ConversationChain(
    llm=llm,
    memory=summary_memory,
#     verbose=True
)

sumamry_memory_chain.invoke({'input' : 'What is my name?'})


In [100]:
end_separator = ""
print_typing_effect("Hallo, selamat datang saya chatbot KoniChat, ada yang bisa saya bantu ? ✋")
while end_separator != "end":
    end_separator = chatting("", chat_history)
# chat_history.clear()

Hallo, selamat datang saya chatbot KoniChat, ada yang bisa saya bantu ? ✋

 halo konichat, perkenalkan saya bento


time=2024-08-14T06:49:46.853Z level=INFO source=sched.go:710 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-7462734796d67c40ecec2ca98eddf970e171dbb6b370e43fd633ee75b69abe1b gpu=GPU-44298984-f28e-4ada-f355-12fda65cafb0 parallel=4 available=15720382464 required="3.3 GiB"
time=2024-08-14T06:49:46.854Z level=INFO source=memory.go:309 msg="offload to cuda" layers.requested=-1 layers.model=27 layers.offload=27 layers.split="" memory.available="[14.6 GiB]" memory.required.full="3.3 GiB" memory.required.partial="3.3 GiB" memory.required.kv="832.0 MiB" memory.required.allocations="[3.3 GiB]" memory.weights.total="1.9 GiB" memory.weights.repeating="1.4 GiB" memory.weights.nonrepeating="461.4 MiB" memory.graph.full="504.5 MiB" memory.graph.partial="965.9 MiB"
time=2024-08-14T06:49:46.855Z level=INFO source=server.go:393 msg="starting llama server" cmd="/tmp/ollama3412442722/runners/cuda_v11/ollama_llama_server --model /root/.ollama/models

INFO [main] build info | build=1 commit="1e6f655" tid="135826122452992" timestamp=1723618186
INFO [main] system info | n_threads=2 n_threads_batch=-1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="135826122452992" timestamp=1723618186 total_threads=4
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="38731" tid="135826122452992" timestamp=1723618186


time=2024-08-14T06:49:47.107Z level=INFO source=server.go:627 msg="waiting for server to become available" status="llm server loading model"
llama_model_loader: - kv  23:                      tokenizer.ggml.scores arr[f32,256000]  = [-1000.000000, -1000.000000, -1000.00...
llama_model_loader: - kv  24:                  tokenizer.ggml.token_type arr[i32,256000]  = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...
llama_model_loader: - kv  25:                tokenizer.ggml.bos_token_id u32              = 2
llama_model_loader: - kv  26:                tokenizer.ggml.eos_token_id u32              = 1
llama_model_loader: - kv  27:            tokenizer.ggml.unknown_token_id u32              = 3
llama_model_loader: - kv  28:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  29:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  30:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - 

...

Halo Bento! Untuk informasi tentang aturan cuti kerja, kamu perlu memperhatikan beberapa hal berikut:

* **Peraturan Cuti:** Perusahaan memiliki peraturan khusus mengenai cuti kerja. Kamu bisa cek di buku panduan perusahaan atau bertanya pada atasanmu. 
* **Jenis Cuti:** Ada berbagai jenis cuti yang tersedia, seperti cuti sakit, cuti melahirkan, cuti religi, dan lain-lain.  
* **Peraturan Waktu:** Setiap jenis cuti memiliki aturan waktu yang berbeda. Kamu perlu memahami aturan tersebut sebelum mengajukan cuti. 
* **Dokumen:** Pastikan kamu memiliki dokumen pendukung untuk setiap jenis cuti, seperti surat keterangan dokter untuk cuti sakit.

Apakah ini membantu, Bento? 😊 




 terima kasih


[GIN] 2024/08/14 - 06:52:30 | 200 |  1.632965187s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/14 - 06:52:31 | 200 |  1.164376903s |       127.0.0.1 | POST     "/api/generate"


Halo Bento!  Ya, penjelasan peraturan kerja itu membantu banget. 😊 

Apakah ini membantu, Bento? 😊 


 end


In [54]:
print(chat_history)

[HumanMessage(content='halo, konichat saya bento'), 'Halo Bento!  Wah, seru sekali kamu mau ngobrol tentang peraturan kerja.  \n\nOke, ini dia penjelasannya:\n\n* **Jam Kerja Lembur:** Kalau hari raya jatuh pada hari kerja, kamu akan dibayar 2 kali upah sejam (7 jam atau 5 jam) jika hari raya jatuh pada hari kerja terpendek dalam 6 hari kerja seminggu. Untuk jam kerja pertama selebihnya 7 jam atau 5 jam, kamu akan dibayar 3 kali upah sejam.  \n* **Jam Kerja Kedua:** Setelah 7 jam atau 5 jam, kamu akan dibayar 4 kali upah sejam jika hari raya jatuh pada hari kerja terpendek dalam 6 hari kerja seminggu. \n* **Sakit dan Upah:** Kalau kamu sakit bulanan dan bisa dibuktikan dengan surat keterangan dokter, maka upah tetap dibayarkan.  Upah akan dibayarkan sesuai ketentuan UU No. 13 tahun 2003 dan PP No 78 Tahun 2015 tentang Pengupahan.\n* **Kecelakaan Kerja:** Kalau kamu tertimpa kecelakaan kerja, kamu bisa mendapatkan bantuan dari BPJS.\n\nApakah ini membantu, Bento? 😊\n\n\n']
